In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
from scipy.stats import linregress

from api_keys import weather_api_key

from citipy import citipy

output_data_file = "output_data/cities.csv"
weather_url = "https://api.openweathermap.org/data/2.5/weather?q="

lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
lat_lngs = []
cities = []

lats = np.random.uniform(low=-90, high=90, size=1500)
lngs = np.random.uniform(low=-180, high=180, size=1500)
lat_lngs = zip(lats, lngs)

for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    if city not in cities:
        cities.append(city)
        
len(cities)

613

In [3]:
# create list of responses
response_list = []

# create counters for output
i = 0
record = 1
group = 1

# new idea
df = {
    "City": [],
    "Cloudiness": [],
    "Country": [],
    "Date": [],
    "Humidity": [],
    "Lat": [],
    "Lng": [],
    "Max Temp": [],
    "Wind Speed": []
}

# initial printout
print(f"Beginning Data Retrieval\n"
      f"-----------------------------")

# for loop for api calls for each city
for city in cities:
    
    # create target_url
    target_url = f"{weather_url}{city}&appid={weather_api_key}"
        
    # generate api request and make json
    response = requests.get(target_url).json()
        
    # create a response string
    response_string = str(response)
        
    if response_string == "{'cod': '404', 'message': 'city not found'}":
        print("City not found. Skipping...")
        
    else:
        # print which city is processing
        print(f"Processing Record {record} of Set {group} | {city}")        
        
        # append response to list of responses
        response_list.append(response)
        
        # new idea
        df["City"].append(response_list[i]['name'])
        df["Cloudiness"].append(response_list[i]['clouds']['all'])
        df["Country"].append(response_list[i]['sys']['country'])
        df["Date"].append(response_list[i]['dt'])
        df["Humidity"].append(response_list[i]['main']['humidity'])
        df["Lat"].append(response_list[i]['coord']['lat'])
        df["Lng"].append(response_list[i]['coord']['lon'])
        df["Max Temp"].append(response_list[i]['main']['temp_max'])
        df["Wind Speed"].append(response_list[i]['wind']['speed'])
        i += 1
    
    # count record and set
    if record < 50:
        record += 1
    else:
        record = 1
        group += 1

# final printout
print(f"-----------------------------\n"
      f"Data Retrieval Complete\n"
      f"-----------------------------")

Beginning Data Retrieval
-----------------------------
Processing Record 1 of Set 1 | anito
Processing Record 2 of Set 1 | faanui
Processing Record 3 of Set 1 | georgetown
Processing Record 4 of Set 1 | ushuaia
Processing Record 5 of Set 1 | kismayo
Processing Record 6 of Set 1 | samarai
Processing Record 7 of Set 1 | north battleford
Processing Record 8 of Set 1 | vestmannaeyjar
Processing Record 9 of Set 1 | richards bay
Processing Record 10 of Set 1 | rikitea
Processing Record 11 of Set 1 | bambous virieux
Processing Record 12 of Set 1 | ardakan
Processing Record 13 of Set 1 | todos santos
Processing Record 14 of Set 1 | hilo
Processing Record 15 of Set 1 | hermanus
City not found. Skipping...
Processing Record 17 of Set 1 | barrow
City not found. Skipping...
Processing Record 19 of Set 1 | bengkulu
City not found. Skipping...
Processing Record 21 of Set 1 | saldanha
Processing Record 22 of Set 1 | narasannapeta
Processing Record 23 of Set 1 | poum
Processing Record 24 of Set 1 | es

In [4]:
# convert data to a data frame
data = pd.DataFrame(df)

# convert from K to F
data["Max Temp"] = round((data["Max Temp"] - 273.15) * (9/5) + 32, 2)
data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Anito,59,PH,1579579213,78,12.45,125.29,78.06,8.93
1,Faanui,21,PF,1579579112,74,-16.48,-151.75,81.95,4.28
2,George Town,20,MY,1579579100,79,5.41,100.34,86.00,2.60
3,Ushuaia,40,AR,1579579078,71,-54.80,-68.30,50.00,4.10
4,Kismayo,0,SO,1579579214,81,-0.36,42.55,80.33,8.99
...,...,...,...,...,...,...,...,...,...
566,Praia,6,CV,1579579300,64,14.92,-23.51,69.80,5.10
567,Belmonte,40,BR,1579579392,78,-15.86,-38.88,78.80,2.60
568,Kintampo,61,GH,1579579392,69,8.06,-1.73,77.20,2.97
569,Port Hedland,7,AU,1579579304,71,-20.32,118.57,93.20,6.70
